In [ ]:
import dis
import inspect
import pprint
import sys
from textwrap import dedent
import math

from codetransformer.utils.pretty import a as show_ast, d as show_disassembly

import pybay2016
from pybay2016 import diagrams

from pybay2016.tokens import show_tokens

# Clean bytecode caches so that importing rot13 fails the first time.
# This is using IPython's magic interpolation syntax.
!rm {pybay2016.__file__.replace('__init__.py', '__pycache__')}/*

# Redraw diagrams
diagrams.draw_all()

 <center>
  <h1>Unspeakably Evil Hacks in Service of Marginally-Improved Syntax</h1><br><br>
  <h2>Compile-Time Metaprogramming in Python</h2><br><br>
  <h3><a href="https://github.com/ssanderson/pytenn2016">https://github.com/ssanderson/pytenn2016</a>
  </h3><br>
  
</center>

# About Me:

<img src="images/me.jpg" alt="Drawing" style="width: 300px;"/>

- API Design Lead at [Quantopian](www.quantopian.com)
- Background in Mathematics and Philosophy
- **Twitter:** [@scottbsanderson](https://twitter.com/scottbsanderson)
- **GitHub:** [ssanderson](github.com/ssanderson)

# Outline

- "Standard" Metaprogramming Techniques
- Python Program Representations
- Custom File Encodings
- Import Hooks
- Bytecode Rewriting
- Conclusions

- Spicy Memes!

# Metaprogramming

> Metaprogramming is the writing of computer programs with the ability to treat programs as their data. It means that a program could be designed to read, generate, analyse or transform other programs, and even modify itself while running. - Wikipedia

<img alt="Xzibit loves metaprogramming!" src="images/yo-dawg.jpg" style="width: 1000px"/>

# Decorators

In [ ]:
def noisy_add(a, b):
    print("add called with args: {args}".format(args=(a, b)))

    return a + b

...

def noisy_save(s):
    print("save called with args: {args}".format(args=(s,)))
    
    # /dev/null is web scale
    with open('/dev/null', 'w') as f:
        f.write(s)

noisy_add(1, 2)
noisy_save('Important Data')

In [ ]:
from functools import wraps

def noisy(f):
    "A decorator that prints arguments to a function before calling it."
    name = f.__name__
    
    @wraps(f)
    def print_then_call_f(*args):
        print("{f} called with args: {args}".format(f=name, args=args))
        return f(*args)
    
    return print_then_call_f

In [ ]:
@noisy
def add(a, b):
    return a + b 

@noisy
def save(s):
    # Still web scale
    with open('/dev/null', 'w') as f:
        f.write(s)

add(1, 2)
save("Important Data")

# Metaclasses

In [ ]:
class Vector:
    "A 2-Dimensional vector."

    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def magnitude(self):
        return math.sqrt(self.x ** 2 + self.y ** 2)
    
    def doubled(self):
        return type(self)(self.x * 2, self.y * 2)
    
v0 = Vector(1, 2) 
print("Magnitude: %f" % v0.magnitude())
print("Doubled Magnitude: %f" % v0.doubled().magnitude())

In [ ]:
class PropertyVector:
    "A 2-Dimensional vector, now with 100% fewer parentheses!"
    def __init__(self, x, y):
        self.x = x
        self.y = y

    @property
    def magnitude(self):
        return math.sqrt(self.x ** 2 + self.y ** 2)
    
    @property
    def doubled(self):
        return type(self)(self.x * 2, self.y * 2)
    
v1 = PropertyVector(1, 2)
print("Magnitude: %f" % v1.magnitude)
print("Doubled Magnitude: %f" % v1.doubled.magnitude)

In [ ]:
# Our metaclass will automatically convert anything with this signature
# into a property.
property_signature = inspect.FullArgSpec(
    args=['self'], varargs=None, varkw=None, defaults=None, 
    kwonlyargs=[], kwonlydefaults=None, annotations={},
)

class AutoPropertyMeta(type):
    """Metaclass that wraps no-argument methods in properties."""
    
    def __new__(mcls, name, bases, clsdict):
        for name, class_attr in clsdict.items():
            try:
                signature = inspect.getfullargspec(class_attr)
            except TypeError:
                continue

            if signature == property_signature:
                print("Wrapping %s in a property." % name)
                clsdict[name] = property(class_attr)

        return super().__new__(mcls, name, bases, clsdict)

In [ ]:
class AutoPropertyVector(metaclass=AutoPropertyMeta):
    "A 2-Dimensional vector, now with 100% less @property calls!"
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def magnitude(self):
        return math.sqrt(self.x ** 2 + self.y ** 2)
    
    def doubled(self):
        return type(self)(self.x * 2, self.y * 2)

v2 = AutoPropertyVector(1, 2)
print("")
print("Magnitude: %f" % v2.magnitude)
print("Doubled Magnitude: %f" % v2.doubled.magnitude)

# `exec`

"The Swiss Army Knife of Metaprogramming"

In [ ]:
from pybay2016.simple_namedtuple import simple_namedtuple

Point = simple_namedtuple('Point', ['x', 'y', 'z'])
p = Point(x=1, y=2, z=3)

print("p.x is {p.x}".format(p=p))
print("p[1] is {p[1]}".format(p=p))

# Review

- Decorators allow us to naturally express modifications to existing classes and functions.
- Metaclasses allow us to customize class creation.
- `exec` allows us to use string-manipulation tools for program manipulation.

![Metaprogramming is the best](images/unicorn.jpg)

# That's all great but...

- Abstractions often incur runtime overhead.

- Certain operations can't be overloaded (e.g. ``is`` and ``not``, or catching exceptions).

- No support for syntactic extensions.
  - Can't add new syntax.
  - Often can't repurpose existing syntax.

- Doesn't quite feel *evil* enough...

<img alt="We have to go deeper!" src="images/leo.jpg" style="width: 1000px"/>

# Let's look at a complicated function...

In [ ]:
def addtwo(a):
    return a + 2

addtwo(1)

# What Happened When I Hit Enter?

<img src="images/compiler.svg" width="3000"/>

In [ ]:
raw_source = b"""\
def addtwo(a):
    return a + 2
    
addtwo(1)
"""

raw_source
list(raw_source)[:10]

In [ ]:
# Bytes to Text
import codecs

decoded_source = codecs.getdecoder('utf-8')(raw_source)[0]

print(decoded_source)

In [ ]:
# Text to AST
import ast
syntax_tree = ast.parse(decoded_source)

body = syntax_tree.body
show_ast(body[1])

In [ ]:
# AST -> Bytecode
code = compile(syntax_tree, 'pybay2016', 'exec')
show_disassembly(code)


<img src="images/scheming.jpg" width="1000"/>

# What can I muck with?
<img src="images/compiler.svg" width="3000"/>

# Custom Source Encodings

<img src="images/decoder.svg" width="2500"/>

In [ ]:
!cat ../pybay2016/rot13.py

### Python doesn't know about our encoding...

In [ ]:
from pybay2016.rot13 import hello 

hello()

### ...until we register with the `codecs` module.

In [ ]:
from codecs import register
from pybay2016.encoding import search_function

register(search_function)

In [ ]:
from pybay2016.rot13 import hello

hello()

# What is this actually useful for?

In [ ]:
!cat ../pybay2016/pyxl.py

In [ ]:
import pyxl.codec.register # Activates the pyxl encoding

from pybay2016.pyxl import hello_html

hello_html()

In [ ]:
str(hello_html())

# Custom Encoding Summary

- Encodings registered globally with the `codecs` module.
- Opt-in on a **per-file** basis.
- Only one encoding per file.
- Files must end in `.py`.
- Operates semantically on bytes <-> text layer.

# Import Hooks

<img src="images/importhook.svg" width="1000"/>

## Hy - Lisp Embedded in Python

In [ ]:
! cat ../pybay2016/hy_example.hy

In [ ]:
from pybay2016.hy_example import hyfact

hyfact(5)

In [ ]:
print("Before:")
pprint.pprint(sys.meta_path[0])

import hy

print("After:")
pprint.pprint(sys.meta_path[0])

In [ ]:
from pybay2016.hy_example import hyfact

hyfact(5)

# Cython - Pseudo-Python Compiled to C

In [ ]:
!cat ../pybay2016/cython_example.pyx

In [ ]:
import pyximport
pyximport.install()

from pybay2016.cython_example import cyfact

print("cyfact is a %s" % type(cyfact))
cyfact(5)

In [ ]:
print("Python Factorial:")
%timeit hyfact(25)

print("\nCython Factorial:")
%timeit cyfact(25)

# Import Hook Summary

- Registered Globally on `sys.meta_path`.
- Loader gets to choose how a file is imported.
- No restrictions on file structure.
  - Doesn't even have correspond to a filesystem entry...

# Problem:

Both import hooks and custom encodings rely on some **external** piece of code having been run before a hooked module can be executed.

This makes it hard to ensure that transformations are used correctly/reliably by users.

# What if we just rewrite the code we already have...?

# Bytecode Manipulation

<img src="images/bytecode.svg" width="750"/>

In [ ]:
addcode = addtwo.__code__
addcode

In [ ]:
from pybay2016.bytecode import code_attrs

code_attrs(addcode)

In [ ]:
import dis

print("Raw Bytes: %s" % list(addcode.co_code))
print("\nDisassembly:\n")
dis.dis(addcode)

### Addition is so 2015...

In [ ]:
def replace_all(l, old, new):
    "Replace all instances of `old` in `l` with `new`"
    out = []
    for elem in l:
        if elem == old: out.append(new)
        else: out.append(elem)
    return out

addbytes = addcode.co_code
mulbytes = bytes(replace_all(list(addbytes), 23, 20))
 
print("Old Disassembly:"); dis.dis(addbytes)
print("\nNew Disassembly:"); dis.dis(mulbytes)

### Just overwriting the code won't work...

In [ ]:
add.__code__.co_code = mulbytes

### ...but copying everything else will!

In [ ]:
from types import CodeType

mulcode = CodeType(
    addcode.co_argcount,
    addcode.co_kwonlyargcount,
    addcode.co_nlocals,
    addcode.co_stacksize,
    addcode.co_flags,
    mulbytes,   # Use our new bytecode.
    addcode.co_consts,
    addcode.co_names,
    addcode.co_varnames,
    addcode.co_filename,
    'multwo',  # Use a new name.
    addcode.co_firstlineno,
    addcode.co_lnotab,
    addcode.co_freevars,
    addcode.co_cellvars,
)

mulcode 

### We can rebuild a modified function the same way.

In [ ]:
from types import FunctionType

multwo = FunctionType(
    mulcode,   # Use new bytecode.
    addtwo.__globals__,
    'multwo',  # Use new __name__.
    addtwo.__defaults__,
    addtwo.__closure__,
)
multwo

In [ ]:
multwo(5)

<img src="images/mindblown.gif" width="2000"/>

# Pattern for Bytecode Transformers

- Start with an existing function.
- Extract its `__code__`.
- Apply some transformation.
- Contruct new `code` by copying everything not changed.
- Construct new function from new code object.

In [ ]:
from codetransformer import CodeTransformer, pattern
from codetransformer.instructions import *

class ruby_strings(CodeTransformer):
    
    @pattern(LOAD_CONST)
    def _format_bytes(self, instr):
        yield instr
        if not isinstance(instr.arg, bytes):
            return

        # Equivalent to:
        # s.decode('utf-8').format(**locals())
        yield LOAD_ATTR('decode')
        yield LOAD_CONST('utf-8')
        yield CALL_FUNCTION(1)
        yield LOAD_ATTR('format')
        yield LOAD_CONST(locals) 
        yield CALL_FUNCTION(0)
        yield CALL_FUNCTION_KW()

In [ ]:
@ruby_strings()
def example(a, b, c):
    return b"a is {a}, b is {b}, c is {c!r}"

example(1, 2, 'foo')

### More Examples:  Overloaded Exceptions

In [ ]:
from codetransformer.transformers.exc_patterns import \
    pattern_matched_exceptions
  
@pattern_matched_exceptions()
def foo():
    try:
        raise ValueError('bar')
    except ValueError('buzz'):
        return 'buzz'
    except ValueError('bar'):
        return 'bar'
    
foo()

### More Examples : Overloaded Literals

In [ ]:
from codetransformer.transformers.literals import ordereddict_literals

@ordereddict_literals
def make_dictionary(a, b):
    return {a: 1, b: 2}

make_dictionary('a', 'b')

# Bytecode Transformer Summary

- Doesn't require external setup.
- Can be applied on a per-function basis like any other decorator.
- Relies on CPython implementation details:
  - Only works in CPython (no PyPy).
  - Significant changes between minor versions.
- Bugs can cause segfaults.

# Conclusions

- Python's standard metaprogramming tools are pretty awesome.
- Import Hooks and Custom Encodings allow us to extend syntax if we're willing to depend on global setup.
- Bytecode transformers give is isolated, composable transformations, at the cost of cross-platform compatibility.

# Thanks!

- **Talk Content:** [https://github.com/ssanderson/pybay2016](https://github.com/ssanderson/pybay2016)
- **Twitter:** [@scottbsanderson](https://twitter.com/scottbsanderson)
- **GitHub:** [ssanderson](github.com/ssanderson)